In [1]:
from pyspark.sql import functions as f

In [2]:
columns = universe.columns

In [3]:
universe = spark.table('dev_derived_ouniverse.output_total_universe_nl')

geoCodeResult = spark.table('data_user_hien.geocoding_universe_nl_result')

noLatLngUniverse = universe.\
                      where((f.col('latitude').isNull()) &(f.col('longitude').isNull())) \
                     .drop('CityGoogle', 'addressGoogle', 'postalCodeGoogle',
                           'latitudeGoogle', 'longitudeGoogle', 'latitude', 'longitude','businessTypeGoogle')
                      

In [4]:
display(geoCodeResult)

In [5]:
geoCodeResult.where(f.col('latitude').isNotNull()).groupBy('partialMatch').count().show()

In [6]:
geoCodeResult = geoCodeResult.\
                       withColumn('operatorId', f.concat(\
                    f.when(f.col('osmId').isNotNull(),\
                           f.col('osmId')).\
                    otherwise(f.lit('')),\
                    f.lit(' / '),\
                    f.when((f.col('placeIdGoogle').isNotNull()),\
                           f.col('placeIdGoogle')).\
                    otherwise(f.lit('')),\
                    f.lit(' / '),\
                    f.when(f.col('ohubID').isNotNull(),\
                           f.col('ohubID')).\
                    otherwise(f.lit(''))))

In [7]:
display(noLatLngUniverse.where(f.col('operatorId') == ' /  / 00079097-34c5-3713-bc36-23055d4d4e19'))

In [8]:
geoCodeResult.where(f.col('latitude').isNotNull()).count()

In [9]:
display(geoCodeResult.where(f.col('operatorId') == ' /  / 00079097-34c5-3713-bc36-23055d4d4e19'))

In [10]:
updateNoLatLngUniverse = noLatLngUniverse.join(geoCodeResult,on = ['operatorId'], how = 'left_outer')
# updateNoLatLngUniverse = updateNoLatLngUniverse.select(columns)

In [11]:
display(updateNoLatLngUniverse.where(f.col('latitude').isNotNull()))

In [12]:
geoCodeResult = spark.table('data_user_hien.geocoding_universe_nl_result').drop('osmid')

In [13]:
geoCodeResult.count()